In [0]:
!pip install git+https://github.com/AlpacaDB/backlight

  Cloning https://github.com/AlpacaDB/backlight to /tmp/pip-req-build-wn41f4fv
    100% |████████████████████████████████| 26.2MB 1.1MB/s 
    100% |████████████████████████████████| 13.9MB 3.0MB/s 
    100% |████████████████████████████████| 12.9MB 3.4MB/s 
    100% |████████████████████████████████| 133kB 31.4MB/s 
    100% |████████████████████████████████| 952kB 19.1MB/s 
    100% |████████████████████████████████| 5.1MB 7.5MB/s 
    100% |████████████████████████████████| 61kB 25.2MB/s 
    100% |████████████████████████████████| 552kB 26.2MB/s 
  Running setup.py bdist_wheel for backlight ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-22o86rum/wheels/95/c2/91/cf62b52737082fc3505586af0619df68c8f3d678bff197bdef
Successfully built backlight
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled

In [0]:
import os
import numpy as np
import pandas as pd
import backlight

# Generate example dummy data

In [0]:
np.random.seed(0)

# market data
if not os.path.exists("example_market.csv"):
    idx = pd.date_range("2018-04-01 00:00:00", "2018-06-30 23:59:59", freq="10S")
    ask = np.cumsum(np.random.rand(len(idx)) - 0.5) + 100.0
    bid = ask - 0.02
    pd.DataFrame(
        index=idx,
        data=np.array([ask, bid]).T,
        columns=["ask", "bid"]
    ).abs().to_csv("example_market.csv")

# signal data
if not os.path.exists("example_signal.csv"):
    sig = np.random.rand(3, len(idx)).T
    n = np.sum(sig, axis=1)
    pd.DataFrame(
        index=idx,
        data=sig / n.reshape(-1, 1),
        columns=["down", "neutral", "up"]
    ).to_csv("example_signal.csv")

# Configuration

In [0]:
model_id = ""
start_dt = "2018-06-03 00:00:00+0000"
end_dt = "2018-06-04 00:00:00+0000"
sig_url = "file:///{}/example_signal.csv".format(os.getcwd())
mkt_url = "file:///{}/example_market.csv".format(os.getcwd())

# loading the signal

In [0]:
from backlight import signal

SYMBOL = "USDJPY"
sig = signal.load_signal(SYMBOL, sig_url, start_dt, end_dt)

In [0]:
sig.tail()

,down,neutral,up,pred
2018-06-03 23:59:20,0.474687,0.394481,0.130832,-1.0
2018-06-03 23:59:30,0.145743,0.038175,0.816082,1.0
2018-06-03 23:59:40,0.605723,0.318978,0.075299,-1.0
2018-06-03 23:59:50,0.431310,0.004492,0.564198,1.0
2018-06-04 00:00:00,0.361087,0.372248,0.266664,0.0


In [0]:
sig.symbol

'USDJPY'

# loading the marketdata

In [0]:
from backlight import datasource

mkt = datasource.load_marketdata(
    sig.symbol,
    sig.start_dt,
    sig.end_dt,
    mkt_url,
)

In [0]:
mkt.tail()

,ask,bid
2018-06-03 23:59:20,17.921237,17.901237
2018-06-03 23:59:30,17.544376,17.524376
2018-06-03 23:59:40,17.266061,17.246061
2018-06-03 23:59:50,16.958572,16.938572
2018-06-04 00:00:00,17.089109,17.069109


# generating labels

## Fixed Neutral

In [0]:
from backlight import labelizer
from backlight.labelizer.ternary.fixed_neutral import FixedNeutralLabelizer


lbl_fix = labelizer.generate_labels(
    mkt,
    FixedNeutralLabelizer(
        lookahead="3Min",
        neutral_atol=0.075,
        neutral_rtol=0.00,
    ))

In [0]:
lbl_fix.head()

,mid,label_diff,label
2018-06-03 00:00:00,80.257283,-0.854094,-1.0
2018-06-03 00:00:10,80.010294,-0.446213,-1.0
2018-06-03 00:00:20,79.701448,0.156435,1.0
2018-06-03 00:00:30,79.822766,-0.402531,-1.0
2018-06-03 00:00:40,79.959499,-0.609441,-1.0


In [0]:
lbl_fix.label_type

'ternary'

In [0]:
# (-1.0, 0.0, 1.0) : (Down, Neutral, Up)
lbl_fix.groupby("label").label.count() / len(lbl_fix)

label
-1.0    0.527485
 0.0    0.044208
 1.0    0.426224
Name: label, dtype: float64

## Dynamic Neutral

In [0]:
from backlight.labelizer.ternary.dynamic_neutral import DynamicNeutralLabelizer

lbl_dyn = labelizer.generate_labels(
    mkt,
    DynamicNeutralLabelizer(
        lookahead="3Min",
        neutral_ratio=0.38,
        neutral_window="1H",
        neutral_hard_limit=0.0,
    ))

In [0]:
lbl_dyn.head()

,label_diff,label,neutral_range
2018-06-03 00:00:00,-0.854094,-1.0,0.854094
2018-06-03 00:00:10,-0.446213,0.0,0.601208
2018-06-03 00:00:20,0.156435,0.0,0.376667
2018-06-03 00:00:30,-0.402531,0.0,0.408647
2018-06-03 00:00:40,-0.609441,-1.0,0.425246


In [0]:
lbl_dyn.label_type

'ternary'

In [0]:
# (-1.0, 0.0, 1.0) : (Down, Neutral, Up)
lbl_dyn.groupby("label").label.count() / len(lbl_dyn)

label
-1.0    0.347876
 0.0    0.379933
 1.0    0.270108
Name: label, dtype: float64

# simulate trading strategy

In [0]:
from backlight import strategies
trades = strategies.simple_entry(mkt, sig, max_holding_time=pd.Timedelta('30min'))

In [0]:
from backlight.trades import flatten

trade = flatten(trades)

In [0]:
trade.tail()

2018-06-03 23:59:20    -2.0
2018-06-03 23:59:30     0.0
2018-06-03 23:59:40    -2.0
2018-06-03 23:59:50     0.0
2018-06-04 00:00:00    11.0
dtype: float64

# simulate and evaluate positions

In [0]:
from backlight import positions
positions = positions.calc_positions(trades, mkt)

In [0]:
positions.head()

,amount,price
2018-06-03 00:00:00,0.0,80.257283
2018-06-03 00:00:10,0.0,80.010294
2018-06-03 00:00:20,1.0,79.701448
2018-06-03 00:00:30,1.0,79.822766
2018-06-03 00:00:40,2.0,79.959499


# Calculate Metrics - based on the raw signals

In [0]:
from backlight import metrics
m = metrics.calc_metrics(sig, lbl_fix)
m

/usr/local/lib/python3.6/dist-packages/backlight/metrics/ternary.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pl.loc[sig.pred == TD.D.value] *= -1


,cnt_uu,cnt_un,cnt_ud,cnt_nu,cnt_nn,cnt_nd,cnt_du,cnt_dn,cnt_dd,cnt_total,hit_ratio,hedge_ratio,neutral_ratio,coverage,avg_pl,total_pl
metrics,1228.0,123.0,1533.0,1191.0,116.0,1498.0,1264.0,143.0,1527.0,8641.0,0.496218,0.519251,0.324615,0.673302,-0.020959,-121.93917


In [0]:
from backlight import metrics
m = metrics.calc_metrics(sig, lbl_dyn)
m

/usr/local/lib/python3.6/dist-packages/backlight/metrics/ternary.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pl.loc[sig.pred == TD.D.value] *= -1


,cnt_uu,cnt_un,cnt_ud,cnt_nu,cnt_nn,cnt_nd,cnt_du,cnt_dn,cnt_dd,cnt_total,hit_ratio,hedge_ratio,neutral_ratio,coverage,avg_pl,total_pl
metrics,772.0,1089.0,1023.0,780.0,1028.0,997.0,782.0,1166.0,986.0,8641.0,0.493404,0.689756,0.324615,0.673302,-0.020959,-121.93917


# Calculate Performance

In [0]:
from backlight import metrics
m = metrics.calc_position_performance(positions)
m

,avg_pl_per_amount,total_pl,total_win_pl,total_lose_pl,cnt_amount
metrics,-0.077857,-594.363405,9441.977049,-10036.340454,7634.0


In [0]:
from backlight import metrics
m = metrics.calc_trade_performance(trades, mkt)
m

,cnt_trade,cnt_win,cnt_lose,win_ratio,lose_ratio,avg_pl_per_amount,total_pl,total_win_pl,total_lose_pl,cnt_amount,avg_win_pl,avg_lose_pl,avg_pl_per_trade
metrics,8641.0,2874.0,2957.0,0.3326,0.342206,-0.077857,-594.363405,9441.977049,-10036.340454,7634.0,3.285309,-3.394096,-0.068784
